# Generate footprint climatology and export results

## Import Supporting Libraries

In [ ]:
import sys
import os
import warnings
import pandas as pd
import configparser

## Import Fluxfootprints

In [2]:
sys.path.insert(0, os.path.abspath("../../src"))  # adjust path as needed

from fluxfootprints import (
    build_climatology,
    summarize_periods, export_contours_gpkg, export_rasters_geotiff
)

## 1. Load site config and half-hourly data

In [ ]:
config_path = "G:/Shared drives/UGS_Flux/Data_Processing/config/US-UTV.ini"
cfg = configparser.ConfigParser(interpolation=None)  
cfg.read(config_path)
lat = cfg.getfloat("METADATA", "station_latitude")
lon = cfg.getfloat("METADATA", "station_longitude")


df = pd.read_csv("G:/Shared drives/UGS_Flux/Data_Processing/config/station_data/US-UTV_HH_202309271630_202507221300.csv")
df['datetime_start'] = pd.to_datetime(df['datetime_start'], format="%Y-%m-%d %H:%M:%S")
df = df.set_index('datetime_start')
df = df.sort_index()
# Instrument & canopy heights (site-level constants if not in file)
df["instr_height_m"] = 3.66          # set your tower’s measurement height here (m)
df["canopy_height_m"] = 0.2         # set your vegetation height here (m)
df["Z0_roughness"] = 0.123 * df["canopy_height_m"]  # roughness length ≈ 0.123*h_c
df['date_time'] = df.index
df['LE'] = df['LE_1_1_1']
df['H'] = df['H_1_1_1']
df['Rn'] = df['NETRAD_1_1_1']
df['G'] = df['G_1_1_A']

## 2. Build the footprint climatology object (computes f_2d per timestep)

In [ ]:

clim = build_climatology(
    df,
    dx=5.0, dy=5.0,
    domain=(-300, 300, -300, 300),   # 500 m domain at 5 m resolution
    inst_height=3.66,
    crop_height=0.5,
    atm_bound_height=2000.0,
)

## 3. Summarize to daily & monthly

In [ ]:
summ = summarize_periods(
    clim, df,
    et_source="LE",        # convert LE W/m² → ET mm/hr for weighting
    daily=True,
    monthly=True
)

c:\Users\paulinkenbrandt\.conda\envs\py313\Lib\site-packages\xarray\core\computation.py:824: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)


## 4. Export 50% and 80% footprint contours to a GeoPackage + CSV stats

In [ ]:
export_contours_gpkg(
    clim, summ, df,
    station_lat=lat,
    station_lon=lon,
    gpkg_path="US-CRT_footprints.gpkg",
    crs_out="auto",            # auto-choose UTM zone for station
    levels=[0.9],         # multiple contour levels
    stats_csv="US-CRT_footprints_stats.csv",
    centroid_out=26912,        # NAD83 / UTM zone 12N
    contour_method="skimage",  # or "rasterio" / "auto"
    rn_col=None, h_col=None,   # optionally set explicit column names
    le_col=None, g_col=None    # (auto-detects if left None)
)

WindowsPath('ffp_geotiffs')

## 5. Export footprint fields as GeoTIFF rasters

In [ ]:

export_rasters_geotiff(
    clim, summ,
    station_lat=lat,
    station_lon=lon,
    out_dir="ffp_geotiffs",
    crs_out="auto",
    which=("daily_mean", "monthly_mean", "daily_etw", "monthly_etw"),
    prefix="US-UTV"
)